In [1]:
#===========================================================================================
# ElasticSearch 텍스트 임베딩 테스트 예제
# 
# =>ElasticSearch 7.3.0 버전부터는 cosine similarity 검색을 지원한다.
# => 데이터로 고차원벡터를 집어넣고, 벡터형식의 데이터를 쿼리(검색어)로 하여 
# 코사인 유사도를 측정하여 가장 유사한 데이터를 찾는다.
# => 여기서는 ElasticSearch와 S-BERT를 이용함
# => ElasticSearch에 index 파일은 index_1.json /데이터 파일은 KorQuAD_v1.0_train_convert.json 참조
#
# => 참고자료 : https://skagh.tistory.com/32
#
# => 참고로 여기서는 title_vector 만 구함, paragrapha_vector는 cpu에서는 엄청 오래 걸려서 주석처리하였음
#===========================================================================================

# sentenceTransformers 라이브러리 설치
#!pip install -U sentence-transformers

# elasticsearch 서버 접속 모듈 설치
# !pip install elasticsearch

# 한국어 문장 분리기(kss) 설치
#!pip install kss

In [2]:
import torch
from sentence_transformers import SentenceTransformer, util
import kss, numpy
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from tqdm.notebook import tqdm

from elasticsearch import Elasticsearch
from elasticsearch import helpers

# elastic 서버 접속 테스트
#es = Elasticsearch("https://192.168.0.91:9200/", verify_certs=False)
#es = Elasticsearch("http://192.168.0.130:9200/")
#es.info()

device = torch.device('cpu')

In [3]:
# s-bert 모델 테스트
sbert_model_path = 'F:\\AI\\model\\sbert-ts2022-04-01-distiluse-7'

# cpu 모델로 실행할때는 device=cpu, 기본은 GPU임
embedder = SentenceTransformer(sbert_model_path, device=device)

text = '나는 오늘 밥을 먹는다.'
vectors = embedder.encode(text, convert_to_tensor=True)
vector_list = [vector.numpy().tolist() for vector in vectors]

print(f'vector_len:{len(vector_list)}')


vector_len:768


In [6]:
# 인덱싱 함수 
def index_data():
    es.indices.delete(index=INDEX_NAME, ignore=[404])
    
    count = 0
        
    # 인덱스 생성
    with open(INDEX_FILE) as index_file:
        source = index_file.read().strip()
      
        count += 1
        print(f'{count}:{source}')
      
        es.indices.create(index=INDEX_NAME, body=source)
        
    count = 0
    
    # DATA 추기
    with open(DATA_FILE) as data_file:
        for line in data_file:
            line = line.strip()
            
            json_data = json.loads(line)
            docs = []
            
            for j in tqdm(json_data):
                count += 1
                
                docs.append(j)
                if count % BATCH_SIZE == 0:
                    index_batch(docs)
                    docs = []
                    print("Indexed {} documents.".format(count))
                  
                # ** 500 개만 보냄
                #if count >= 500:
                #    break
                    
            if docs:
                index_batch(docs)
                print("Indexed {} documents.".format(count))
                    
    es.indices.refresh(index=INDEX_NAME)
    print("=== End Done indexing===")
    
    
# 문단(paragraph)들 분리
# 문장으로 나누고, 해당 vector들의 평균을 구함.
# =>굳이 elasticsearch에 문단 벡터는 추가하지 않고, title 벡터만 이용해도 되므로 주석처리함
'''
def paragraph_index(paragraph):
    avg_paragraph_vec = numpy.zeros((1,768))
    sent_count = 0
    
    #print(paragraph)
    # kss로 분할할때 줄바꿈 있으면, 파싱하는데 에러남.따라서 "\n"는 제거함
    paragraph = paragraph.replace("\n","")
    
    #print("==Start paragraph_index==")
    for sent in kss.split_sentences(paragraph):
        # 문장으로 나누고, 해당 vector들의 평균을 구함.
        avg_paragraph_vec += embed_text([sent])
        sent_count += 1
            
    avg_paragraph_vec /= sent_count
    return avg_paragraph_vec.ravel(order='C')
'''

def index_batch(docs):
   
    titles = [doc["title"] for doc in docs]
    title_vectors = embed_text(titles)
    
    # 문장이 길면 분할해서 embedding을 구해야 하는데, 여기서는 분할하지 않고 embedding을 구함
    paragraphs = [doc["paragraph"] for doc in docs]
    paragraph_vectors = embed_text(paragraphs)
    
    # * cpu로 문단은 임베딩하는데 너무 오래 걸리므로 주석처리함
    #paragraph_vectors = [paragraph_index(doc["paragraph"]) for doc in tqdm(docs)]
    requests = []
    
    for i, doc in enumerate(tqdm(docs)):
        
        #request["title"] = doc["title"]           # 이렇게 등록은 안됨
        #request["paragraph"] = doc["paragraph"]
        
        request = doc
        
        request["_op_type"] = "index"
        request["_index"] = INDEX_NAME
        
        request["title_vector"] = title_vectors[i]
        request["paragraph_vector"] = paragraph_vectors[i]
        #request["paragraph_vector"] = paragraph_vectors[i]  # * cpu로 문단은 임베딩하는데 너무 오래 걸리므로 주석처리함
        
        requests.append(request)
        
    # batch 단위로 한꺼번에 es에 데이터 insert 시킴     
    bulk(es, requests)
    
# embedding 모델에서 vector를 구함    
def embed_text(input):
    vectors =  embedder.encode(input, convert_to_tensor=True)
    return [vector.numpy().tolist() for vector in vectors]
          

In [7]:
#======================================================================================
# ElasticSearch(이하:ES) 데이터 인텍싱
# - ElasticSearch(이하:ES)에 KorQuAD_v1.0_train_convert.json 파일에 vector값을 구하여 index 함
#
# => index 명 : korquad
# => index 구조 : index_1.json 파일 참조
# => BATCH_SIZE : 100 => 100개의 vector값을 구하여, 한꺼번에 ES에 인텍스 데이터를 추가함.
#======================================================================================
INDEX_NAME = 'korquad'
INDEX_FILE = './data/index.json'
DATA_FILE = './data/KorQuAD_v1.0_train_convert.json'
BATCH_SIZE = 100

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.130:9200/")
print(es.info())

# 2. index 처리
index_data()

{'name': 'lenovo-x240', 'cluster_name': 'mpower', 'cluster_uuid': 'JJ1h3dNTRvSLJFW3gj5MCw', 'version': {'number': '7.17.3', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '5ad023604c8d7416c9eb6c0eadb62b14e766caff', 'build_date': '2022-04-19T08:11:19.070913226Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}
1:{
  "settings": {
    "number_of_shards": 2,
    "number_of_replicas": 1
  },
   "mappings": {
    "dynamic": "true",
    "_source": {
      "enabled": "true"
    },
    "properties": {
      "title": {
        "type": "text"
      },
	  "paragraph": {
        "type": "text"
      },
      "title_vector": {
        "type": "dense_vector",
        "dims": 768
      },
	  "paragraph_vector": {
        "type": "dense_vector",
        "dims": 768
      }
    }
  }
}


  0%|          | 0/1420 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Indexed 100 documents.


  0%|          | 0/100 [00:00<?, ?it/s]

...

Indexed 1300 documents.


  0%|          | 0/100 [00:00<?, ?it/s]

Indexed 1400 documents.


  0%|          | 0/20 [00:00<?, ?it/s]

Indexed 1420 documents.
=== End Done indexing===


In [ ]:
# kibana 콘솔창에 접속해서 계수 확인
# http://192.168.0.130:5601/app/dev_tools 에 접속해서 해야함

## 입력 ##
# GET korquad/_count

## 출력 ###
'''
{
  "count" : 1420,
  "_shards" : {
    "total" : 2,
    "successful" : 2,
    "skipped" : 0,
    "failed" : 0
}
'''    

In [8]:
# 검색 하기

import time
from elasticsearch import Elasticsearch

def run_query_loop():
    while True:
        try:
            handle_query()
        except KeyboardInterrupt:
            return
        
def handle_query():
    
    query = input("검색어 입력: ")
    
    start_embedding_time = time.time()
    query_vector = embed_text([query])[0]
    end_embedding_time = time.time() - start_embedding_time
    
    # 쿼리 구성
    script_query = {
        "script_score":{
            "query":{
                "match_all": {}},
            "script":{
                "source": "cosineSimilarity(params.query_vector, doc['paragraph_vector']) + 1.0",  # 뒤에 1.0 은 코사인유사도 측정된 값 + 1.0을 더해준 출력이 나옴
                "params": {"query_vector": query_vector}
            }
        }
    }
    
    #print('query\n')
    #print(script_query)
    
    # 실제 ES로 검색 쿼리 날림
    start_search_time = time.time()
    response = es.search(
        index=INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source":{"includes": ["title", "paragraph"]}
        }
    )
    end_search_time = time.time() - start_search_time
    
    print("{} total hits.".format(response["hits"]["total"]["value"])) 
    print("embedding time: {:.2f} ms".format(end_embedding_time * 1000)) 
    print("search time: {:.2f} ms".format(end_search_time * 1000)) 
    print('\n')
    
    # 쿼리 응답 결과값에서 _id, _score, _source 등을 뽑아냄
    # print(response)
    
    for hit in response["hits"]["hits"]: 
        
        print("index:{}, type:{}".format(hit["_index"], hit["_type"]))
        print("id: {}, score: {}".format(hit["_id"], hit["_score"])) 
        
        print(f'[제목] {hit["_source"]["title"]}')
        
        print('[내용]')
        print(hit["_source"]["paragraph"]) 
        
        print()
    

In [19]:
#====================================================================
# ES 인덱싱된 내용 검색 
# => cosineSimilarity 스크립트를 이용하여 ES로 query 함(*이때 SEARCH_SIZE를 몇개로 할지 지정할수 있음)
# => 쿼리 응답 결과 값에서 _id, _score, _source 등을 뽑아냄
#====================================================================

INDEX_NAME = 'korquad'
SEARCH_SIZE = 3

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.130:9200/")
print(es.info)

# 2. query 처리
run_query_loop()

<bound method Elasticsearch.info of <Elasticsearch([{'host': '192.168.0.130', 'port': 9200}])>>


검색어 입력:  제주도


1424 total hits.
embedding time: 21.00 ms
search time: 51.98 ms


index:korquad, type:_doc
id: 3qvIz4AB9i71yjP6BLUw, score: 1.2551259
[제목] 무쓰_국
[내용]
무쓰 국(일본어: 陸奥国 무쓰노쿠니)는 일본의 옛 구니이다. 중국풍으로 줄여서 오슈(奥州)로도 불린다. 《엔기시키》에는 대국(大國)·원국(遠國)으로 격이 분류되었다. 야마토 조정이 위치해 있던 기나이(畿內)에서 보아 산도(山道, 훗날의 도산도)와 해도(海道, 후의 도카이도)의 안쪽에 위치해 있으며, 조정이 새롭게 복종시킨 지역을 토고쿠에 포함시켰기에 시기에 따라 범위가 변천했다. 대체로 혼슈(本州) 북동부에 해당하는 오늘의 후쿠시마(福島) 현, 미야기(宮城) 현, 이와테(岩手) 현, 아오모리(靑森) 현과 아키타(秋田) 현 북동쪽의 카즈노 시와 오사카쵸에 해당하지만, 보신전쟁에 패전한 무츠·에치고 등 여러 쿠니의 동맹제번에 대한 처분 때인 메이지 원년 12월 7일(서기 1869년 1월 19일)에 데와노쿠니와 함께 분할되어 아오모리 현과 이와테 현 니노헤군에 걸친 지역으로 줄어들었다.
처음에는 이름을 미치노오쿠(道奧)라 했다. 《히타치 국 풍토기(常陸國風土記)》에는 고토쿠 천황(孝德天皇) 재위 말년인 654년에 아시가라(足柄) 고개의 동쪽에 히타치 국(常陸國)를 비롯한 여덟 개의 쿠니를 두었다는 기술이 있는데, 이중에 미치노오쿠가 포함되었던 것으로 보인다. 지금의 일본 도호쿠 지방에서 차츰 일본으로 편입된 구역, 즉 미야기 현 남부까지의 넓은 영역을 잠정적으로 포괄한 변경의 대국이었던 히타치에서 분할되는 형태로 성립된 이래로 헤이안 시대까지 미치노쿠(陸奧)로 불렸다. 7세기의 처음 설치했을 당시의 범위는 대체로 지금의 일본 미야기 현 중남부, 야마가타 현의 내륙부 및 후쿠시마 현 거의 대부분 지역에 해당하며, 태평양 쪽뿐 아니라 오우(奧羽) 산맥 서쪽 즉 동해와 마주한 지금의 야마가타 현 내륙 분지군이

In [18]:
#==============================================================================================
# ES index에 데이터 추가하가
# => 추가할 데이터는 {'paragraph': 내용, 'title': 제목} 기존 입려된 방식대로(사전) 입력 되어야 함
#===============================================================================================

# ES에 이미 생성된 index
INDEX_NAME = 'korquad'
BATCH_SIZE = 30


# 1.추가할 데이터 준비
title = [
    '제주도', 
    '한라산',
    '서울특별시'
        ]

paragraph = [
    '대한민국의 남서쪽에 있는 섬. 행정구역상 광역자치단체인 제주특별자치도의 관할. 한국의 섬 중에서 가장 크고 인구가 많은 섬으로 면적은 1833.2㎢이다. 제주도 다음 2번째 큰 섬인 거제도의 5배 정도 된다. 인구는 약 68만 명.',
    '대한민국에서 가장 큰 섬인 제주도에 있으며 대한민국의 실효지배 영토 내의 최고봉이자 가장 높은 산(해발 1,947m). 대한민국의 국립공원 중 하나이다. 국립공원 전역이 유네스코 세계유산으로 지정되었다.',
    '대한민국의 수도인 서울은 현대적인 고층 빌딩, 첨단 기술의 지하철, 대중문화와 예것이 공존하는 대도시. 주목할 만한 명소로는 초현대적 디자인의 컨벤션 홀인 동대문디자인플라자, 한때 7,000여 칸의 방이 자리하던 경복궁, 조계사가 있다',
            ]

# {'paragraph': "", 'title': ""}

# 2. elasticsearch 접속
es = Elasticsearch("http://192.168.0.130:9200/")
print(es.info)

doc = {}
docs = []
count = 0

# 3. batch 사이즈 만큼식 ES에 추가
# => 추가할 데이터는 {'paragraph': 내용, 'title': 제목} 기존 입려된 방식대로(사전) 입력 되어야 함
for title, paragraph in zip(title, paragraph):
    doc = {}
    doc['paragraph'] = paragraph
    doc['title'] = title
    docs.append(doc)
    count += 1
    if count % BATCH_SIZE == 0:
        index_batch(docs)
        docs = []
        print("Indexed {} documents.".format(count))
   
# docs 이 있으면 전송
if docs:
    index_batch(docs)
    print("Indexed {} documents(end).".format(count))


<bound method Elasticsearch.info of <Elasticsearch([{'host': '192.168.0.130', 'port': 9200}])>>


  0%|          | 0/3 [00:00<?, ?it/s]

Indexed 3 documents(end).


In [ ]:
#==============================================================================================
# ES 데이터 조회하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.130:9200/")
print(es.info)

###########################################################
# 인덱스내 데이터 조회 => query 이용
###########################################################
def search(index, data=None):
    if data is None: #모든 데이터 조회
        data = {"match_all":{}}
    else:
        data = {"match": data}
        
    body = {"query": data}
    res = es.search(index=index, body=body)
    return res
###########################################################

# 모든 데이터 조회
#sr = search(index=INDEX_NAME)
#pprint.pprint(sr)

# 단일 필드 조회
sr = search(index=INDEX_NAME, data = {'title': '제주도'})
print(sr)


In [17]:
#==============================================================================================
# ES index에 데이터 삭제하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.130:9200/")
print(es.info)

############################################################
## 1: 인덱스 내의 데이터 삭제 => query 이용
############################################################
def delete(index, data):
    if data is None:  # data가 없으면 모두 삭제
        data = {"match_all":{}}
    else:
        data = {"match": data}
        
    body = {"query": data}
    return es.delete_by_query(index, body=body)

############################################################
## 2: 인덱스 내의 데이터 삭제 => id 이용
############################################################
def delete_by_id(index, id):
    return es.delete(index, id=id)

############################################################
## 3: 인덱스 자체 삭제
############################################################
def delete_index(index):
    if es.indices.exists(index=index):
        return es.indices.delete(index=index)


# 1: query 이용 데이터 삭제
delete(index=INDEX_NAME, data={'title':'한라산'})

# 3: 인덱스 자체 삭제
#delete_index(index=INDEX_NAME)


<bound method Elasticsearch.info of <Elasticsearch([{'host': '192.168.0.130', 'port': 9200}])>>


{'took': 45,
 'timed_out': False,
 'total': 2,
 'deleted': 2,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [65]:
#==============================================================================================
# ES index에 데이터 업데이트하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.130:9200/")
print(es.info)

############################################################
## 1: 인덱스 내의 데이터 업데이트=>_id 에 데이터 업데이트
############################################################
def update(index, id, doc, doc_type):
    
    body = {
        'doc': doc
    }
    
    res=es.update(index=index, id=id, body=body, doc_type=doc_type)
    return res
############################################################

#=====================================================================
# 검색해서, _id, _type을 구함
sr = search(index=INDEX_NAME, data = {'title': '제주도'})

print('\n')
print("===[검색 결과]===")
for hits in sr["hits"]["hits"]:
    id = hits["_id"]      # id
    type = hits["_type"]  # type
    
    print(f'id: {id}')
    print(f'type: {type}')
    print(f'title:{hits["_source"]["title"]}')
    print(f'paragraph:{hits["_source"]["paragraph"]}')
    print('\n')
    
    # update 시킴
    print("===[업데이트]===")
    doc = {'paragraph': '제주도는 대한민국에 가장 남쪽에 있는 섬으로, 인구는 약 71만명이며, 화산섬으로 관광자원이 많은 천혜의 관광지 이다.'}
    print(doc)
    print('\n')
    
    ur=update(index=INDEX_NAME, id=id, doc=doc, doc_type=type)
    print("===[업데이트 결과]===")
    print(ur)
    print('\n')

#=====================================================================

# 인덱스 refresh 함
# elasticsearch의 자동 새로고침의 시간은 1초 정도 소요
# 따라서 코드에 아래 명령어를 입력하지 않았을 경우 검색을 하지 못할 가능성도 존재
es.indices.refresh(index=INDEX_NAME)

# 제주도로 검색해서 한번더 확인
sr = search(index=INDEX_NAME, data = {'title': '제주도'})

print("===[재검색 결과]===")
for hits in sr["hits"]["hits"]:
    
    print(f'id:{hits["_id"]}')
    print(f'type: {hits["_type"]}')
    print(f'title:{hits["_source"]["title"]}')
    print(f'paragraph:{hits["_source"]["paragraph"]}')
    
              

<bound method Elasticsearch.info of <Elasticsearch([{'host': '192.168.0.130', 'port': 9200}])>>


===[검색 결과]===
id: CqvSz4AB9i71yjP66r3A
type: _doc
title:제주도
paragraph:제주도는 대한민국에 가장 남쪽에 있는 섬으로, 인구는 약 70만명이며, 화산섬으로 관광자원이 많은 천혜의 관광지 이다.


===[업데이트]===
{'paragraph': '제주도는 대한민국에 가장 남쪽에 있는 섬으로, 인구는 약 71만명이며, 화산섬으로 관광자원이 많은 천혜의 관광지 이다.'}


===[업데이트 결과]===
{'_index': 'korquad', '_type': '_doc', '_id': 'CqvSz4AB9i71yjP66r3A', '_version': 6, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 710, '_primary_term': 2}


===[재검색 결과]===
id:CqvSz4AB9i71yjP66r3A
type: _doc
title:제주도
paragraph:제주도는 대한민국에 가장 남쪽에 있는 섬으로, 인구는 약 71만명이며, 화산섬으로 관광자원이 많은 천혜의 관광지 이다.
